In [1]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 231228") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .getOrCreate()

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
24/01/03 10:51:25 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
24/01/03 10:51:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kjh/app/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kjh/.ivy2/cache
The jars for the packages stored in: /home/kjh/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3194858c-f925-4e02-a723-921a27aea211;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;1.0.2 in central
	found com.microsoft.azure#synapseml-core_2.12;1.0.2 in central
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found commons-lang#commons-lang;2.6 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in centra

In [2]:
# 복잡한 JSON 파일을 읽을 때는 multiline 옵션을 쓰자

df_bike = spark.read.option('multiline', 'true').json('file:///home/kjh/data/bike/bike.json')
df_weather = spark.read.option('multiline', 'true').json('file:///home/kjh/data/bike/json_cycle_finale.json')

24/01/03 10:51:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
df_bike.show()

24/01/03 10:51:39 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
    !Exec <FileSourceScanExec> cannot run on GPU because JSON input and output has been disabled. To enable set spark.rapids.sql.format.json.enabled to true; unsupported data types StructType(StructField(RESULT,StructType(StructField(CODE,StringType,true),StructField(MESSAGE,StringType,true)),true),StructField(list_total_count,StringType,true),StructField(row,ArrayType(StructType(StructField(END_INDEX,LongType,true),StructField(RENT_CNT,StringType,true),StructField(RENT_NM,StringType,true),StructField(RTN_CNT,StringType,true),StructField

+------------------------+
|               useStatus|
+------------------------+
|{{INFO-000, 정상 처리...|
+------------------------+



In [4]:
from pyspark.sql.functions import explode

# 중첩된 JSON 구조 접근 및 explode 함수 사용
df_bike = df_bike.select(explode("useStatus.row").alias("items"))
df_weather = df_weather.select(explode("response.body.items.item").alias("items"))

In [5]:
# items 컬럼의 필드를 별도의 컬럼으로 펼침
df_bike = df_bike.select("items.*")
df_weather = df_weather.select("items.*")

In [6]:
df_bike.createOrReplaceTempView('bike')
df_weather.createOrReplaceTempView('weather')

In [7]:
df_bike.show()

24/01/03 10:51:50 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
          !Exec <FileSourceScanExec> cannot run on GPU because JSON input and output has been disabled. To enable set spark.rapids.sql.format.json.enabled to true; unsupported data types StructType(StructField(RESULT,StructType(StructField(CODE,StringType,true),StructField(MESSAGE,StringType,true)),true),StructField(list_total_count,StringType,true),StructField(row,ArrayType(StructType(StructField(END_INDEX,LongType,true),StructField(RENT_CNT,StringType,true),StructField(RENT_NM,StringType,true),StructField(RTN_CNT,StringType,true),Struc

+---------+--------+-------------------------------+-------+-----------+----------+-------+
|END_INDEX|RENT_CNT|                        RENT_NM|RTN_CNT|START_INDEX| STAT_DATA|STA_LOC|
+---------+--------+-------------------------------+-------+-----------+----------+-------+
|        0|      16|     729. 서부식자재마트 건너편|     12|          0|2020-01-01| 양천구|
|        0|      20|   731. 서울시 도로환경관리센터|     20|          0|2020-01-01| 양천구|
|        0|       9|                732. 신월중학교|      2|          0|2020-01-01| 양천구|
|        0|       6|       733. 신정이펜하우스314동|      4|          0|2020-01-01| 양천구|
|        0|      13|           734. 신트리공원 입구|     16|          0|2020-01-01| 양천구|
|        0|      11|              735. 영도초등학교|     17|          0|2020-01-01| 양천구|
|        0|       8|                736. 오솔길공원|      9|          0|2020-01-01| 양천구|
|        0|       5|                  737. 장수공원|      5|          0|2020-01-01| 양천구|
|        0|      18|        504. 신자초교입구교차로|     21|          0|2020-

In [8]:
# 필요없어 보이는 컬럼 삭제
df_bike = spark.sql("""
                    SELECT 
                        RENT_NM, 
                        CAST(RENT_CNT AS INT) AS RENT_CNT, 
                        CAST(RTN_CNT AS INT) AS RTN_CNT, 
                        unix_timestamp(STAT_DATA, 'yyyy-MM-dd') AS STAT_DATA
                    FROM bike
                    """)
df_weather = df_weather.drop('iscs', 'stnNm')

In [9]:
# 컬럼의 형식변환
# tm 컬럼은 String으로 된 YYYY-MM-DD 날짜 형식. VectorAssembler 을 사용하기 위해서는 숫자로 변환 필요
# Date 로 먼저 변환한 뒤, Timestamp 형식으로 변환 => 최종 long 타입

from pyspark.sql.functions import col, unix_timestamp, to_date

df_wt = df_weather

# 'tm' 컬럼을 날짜 타입으로 변환하고, 이를 Unix timestamp로 변환
df_wt = df_wt.withColumn('tm', unix_timestamp(to_date(col('tm'), 'yyyy-MM-dd')))

data_types = {
    "stnId": "int",  # 지점 번호는 정수
    "avgM15Te": "float",  # 지중온도는 소수점을 포함할 수 있으므로 float
    "n99Rn": "float",  # 강수량은 소수점을 포함할 수 있음
    "minPs": "float",  # 기압은 소수점을 포함할 수 있음
    "avgRhm": "float",  # 상대습도는 소수점을 포함할 수 있음
    "minRhmHrmt": "int",  # 시각은 정수로 처리
    "maxInsWsWd": "int",  # 풍향은 정수로 처리
    "avgTs": "float",  # 지면온도는 소수점을 포함할 수 있음
    "maxInsWsHrmt": "int",  # 시각은 정수로 처리
    "ddMesHrmt": "int",  # 시각은 정수로 처리
    "maxPsHrmt": "int",  # 시각은 정수로 처리
    "avgPv": "float",  # 증기압은 소수점을 포함할 수 있음
    "minRhm": "float",  # 상대습도는 소수점을 포함할 수 있음
    "sumSsHr": "float",  # 일조 시간은 소수점을 포함할 수 있음
    "ssDur": "float",  # 가조시간은 소수점을 포함할 수 있음
    "avgPs": "float",  # 해면기압은 소수점을 포함할 수 있음
    "maxWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "avgCm5Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "minTg": "float",  # 초상온도는 소수점을 포함할 수 있음
    "maxWsWd": "int",  # 풍향은 정수로 처리
    "sumSmlEv": "float",  # 증발량은 소수점을 포함할 수 있음
    "avgTca": "float",  # 전운량은 소수점을 포함할 수 있음
    "hr1MaxIcsr": "float",  # 일사량은 소수점을 포함할 수 있음
    "avgTd": "float",  # 이슬점온도는 소수점을 포함할 수 있음
    "maxPs": "float",  # 해면기압은 소수점을 포함할 수 있음
    "avgCm20Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "ddMes": "float",  # 적설량은 소수점을 포함할 수 있음
    "minTa": "float",  # 기온은 소수점을 포함할 수 있음
    "minPsHrmt": "int",  # 시각은 정수로 처리
    "avgM50Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "maxTa": "float",  # 기온은 소수점을 포함할 수 있음
    "hr24SumRws": "int",  # 풍정합은 정수로 처리
    "avgM30Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgCm10Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgM05Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "hr1MaxIcsrHrmt": "int",  # 시각은 정수로 처리
    "maxInsWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "avgTca": "float",  # 전운량은 소수점을 포함할 수 있음
    "avgCm30Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgM10Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "sumGsr": "float",  # 일사량은 소수점을 포함할 수 있음
    "maxWsHrmt": "int",  # 시각은 정수로 처리
    "avgPa": "float",  # 현지기압은 소수점을 포함할 수 있음
    "avgWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "sumFogDur": "float",  # 안개 지속 시간은 소수점을 포함할 수 있음
    "sumLrgEv": "float",  # 증발량은 소수점을 포함할 수 있음
    "sumDpthFhsc": "float",  # 신적설은 소수점을 포함할 수 있음
    "ddMefs": "float",  # 적설량은 소수점을 포함할 수 있음
    "ddMefsHrmt": "int",  # 시각은 정수로 처리
    "sumRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "hr1MaxRnHrmt": "int",  # 시각은 정수로 처리
    "hr1MaxRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "mi10MaxRnHrmt": "int",  # 시각은 정수로 처리
    "mi10MaxRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "avgTa": "float",  # 기온은 소수점을 포함할 수 있음
    "minTaHrmt": "int",  # 시각은 정수로 처리
    "maxTaHrmt": "int",  # 시각은 정수로 처리
    "maxWd": "int",  # 풍향은 정수로 처리
    "avgLmac": "float",  # 중하층운량은 소수점을 포함할 수 있음
    "sumRnDur": "float"
}

# 딕셔너리 형태의 컬럼 타입정보를 가지고 withColumn을 for문으로 돌림
for column, data_type in data_types.items():
    df_wt = df_wt.withColumn(column, col(column).cast(data_type))

In [10]:
# 결측치 컬럼 제거
drop_cols = ['ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'mi10MaxRn', 'mi10MaxRnHrmt', 'n99Rn', 'sumDpthFhsc', 'sumFogDur', 'sumRn', 'sumRnDur']
df_wt = df_wt.drop(*drop_cols)

# 결측 데이터 있는 행 제거
df_wt = df_wt.dropna()

In [11]:
# 캐시를 수행하여 아래의 JOIN 작업의 속도를 최적화

df_bike.createOrReplaceTempView('bike')
df_wt.createOrReplaceTempView('wt')
df_bike.cache()
df_wt.cache()

24/01/03 10:51:53 WARN GpuOverrides: 
!Exec <ProjectExec> cannot run on GPU because not all expressions can be replaced
  @Expression <Alias> items#13.RENT_NM AS RENT_NM#22 could run on GPU
    @Expression <GetStructField> items#13.RENT_NM could run on GPU
      @Expression <AttributeReference> items#13 could run on GPU
  @Expression <Alias> cast(items#13.RENT_CNT as int) AS RENT_CNT#190 could run on GPU
    @Expression <Cast> cast(items#13.RENT_CNT as int) could run on GPU
      @Expression <GetStructField> items#13.RENT_CNT could run on GPU
        @Expression <AttributeReference> items#13 could run on GPU
  @Expression <Alias> cast(items#13.RTN_CNT as int) AS RTN_CNT#191 could run on GPU
    @Expression <Cast> cast(items#13.RTN_CNT as int) could run on GPU
      @Expression <GetStructField> items#13.RTN_CNT could run on GPU
        @Expression <AttributeReference> items#13 could run on GPU
  @Expression <Alias> unix_timestamp(items#13.STAT_DATA, yyyy-MM-dd, Some(Asia/Seoul), false) 

DataFrame[avgCm10Te: float, avgCm20Te: float, avgCm30Te: float, avgCm5Te: float, avgLmac: float, avgM05Te: float, avgM10Te: float, avgM15Te: float, avgM30Te: float, avgM50Te: float, avgPa: float, avgPs: float, avgPv: float, avgRhm: float, avgTa: float, avgTca: float, avgTd: float, avgTs: float, avgWs: float, hr1MaxIcsr: float, hr1MaxIcsrHrmt: int, hr24SumRws: int, maxInsWs: float, maxInsWsHrmt: int, maxInsWsWd: int, maxPs: float, maxPsHrmt: int, maxTa: float, maxTaHrmt: int, maxWd: int, maxWs: float, maxWsHrmt: int, maxWsWd: int, minPs: float, minPsHrmt: int, minRhm: float, minRhmHrmt: int, minTa: float, minTaHrmt: int, minTg: float, ssDur: float, stnId: int, sumGsr: float, sumLrgEv: float, sumSmlEv: float, sumSsHr: float, tm: bigint]

In [12]:
df_fin = spark.sql("""
                   SELECT b.*, w.*
                   FROM bike b
                   INNER JOIN wt w
                   ON b.STAT_DATA = w.tm
                   """)

df_fin = df_fin.drop("tm")

In [13]:
df_fin.show()

24/01/03 10:51:53 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
        !Exec <InMemoryTableScanExec> cannot run on GPU because ParquetCachedBatchSerializer is not being used
          @Expression <AttributeReference> RENT_NM#22 could run on GPU
          @Expression <AttributeReference> RENT_CNT#190 could run on GPU
          @Expression <AttributeReference> RTN_CNT#191 could run on GPU
          @Expression <AttributeReference> STAT_DATA#192L could run on GPU
          @Expression <IsNotNull> isnotnull(STAT_DATA#192L) could run on GPU
            @Expression <AttributeReference> STAT_DATA#192L coul

+-------------------------------+--------+-------+----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+------------+------------+-----------+-----------+--------------+----------+-----------+------------+----------+-----------+---------+-----------+---------+-----+-----------+---------+-------+-----------+---------+------+----------+-----+---------+------------+-----------+-----+-----------+-----------+-----------+-----------+
|                        RENT_NM|RENT_CNT|RTN_CNT| STAT_DATA|  avgCm10Te|  avgCm20Te|  avgCm30Te|    avgCm5Te|    avgLmac|   avgM05Te|   avgM10Te|   avgM15Te|   avgM30Te|   avgM50Te|      avgPa|      avgPs|      avgPv|     avgRhm|       avgTa|     avgTca|       avgTd|       avgTs|      avgWs| hr1MaxIcsr|hr1MaxIcsrHrmt|hr24SumRws|   maxInsWs|maxInsWsHrmt|maxInsWsWd|      maxPs|maxPsHrmt|      maxTa|maxTaHrmt|maxWd|

In [14]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import regexp_replace
from synapse.ml.lightgbm import LightGBMRegressor

# 예시: 'RENT_NM' 컬럼의 특수 문자를 '_'로 대체
df_fin = df_fin.withColumn('RENT_NM', regexp_replace('RENT_NM', "[ ,:;{}()\[\]\\n]", "_"))

# StringIndexer를 사용하여 범주형 컬럼을 숫자형으로 변환
stringIndexer = StringIndexer(inputCol="RENT_NM", outputCol="indexed_RENT_NM", handleInvalid="keep")

# OneHotEncoder를 사용하여 숫자 인덱스를 원-핫 인코딩된 벡터로 변환
oneHotEncoder = OneHotEncoder(inputCols=["indexed_RENT_NM"], outputCols=["encoded_RENT_NM"])

# 데이터프레임의 모든 컬럼을 특성으로 사용하고자 할 때
features = df_fin.columns

# 라벨 컬럼과 원본 범주형 컬럼 제외
features.remove('RENT_CNT')
features.remove('RENT_NM')

# StringIndexer와 OneHotEncoder의 출력 컬럼 추가
features.append('encoded_RENT_NM')

# VectorAssembler 생성
assembler = VectorAssembler(
    inputCols=features,
    outputCol="features"
)

# 파이프라인 생성 및 모델 학습
pipeline = Pipeline(stages=[stringIndexer, oneHotEncoder, assembler])
transformer = pipeline.fit(df_fin)
tf_df = transformer.transform(df_fin).select('RENT_CNT', 'features')

# train, test 쪼개기
train, test = tf_df.randomSplit([0.7, 0.3], seed=42)

# # train, test 쪼개기
# train, test = df_fin.randomSplit([0.7, 0.3], seed=42)

# # 모델 생성 및 훈련
# lgbm = LightGBMRegressor(objective='regression',
#                             featuresCol='features',
#                             labelCol='RENT_CNT',
#                             alpha=0.3,
#                             learningRate=0.3,
#                             numIterations=100,
#                             numLeaves=31)

# # 파이프라인 생성 및 모델 학습
# pipeline = Pipeline(stages=[stringIndexer, oneHotEncoder, assembler, lgbm])
# model = pipeline.fit(train)
# pred = model.transform(test).select('RENT_CNT', 'features', 'prediction')

<>:7: SyntaxWarning: invalid escape sequence '\['
<>:7: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_133738/3073691433.py:7: SyntaxWarning: invalid escape sequence '\['
  df_fin = df_fin.withColumn('RENT_NM', regexp_replace('RENT_NM', "[ ,:;{}()\[\]\\n]", "_"))
24/01/03 10:52:05 WARN GpuOverrides: 
!Exec <ObjectHashAggregateExec> cannot run on GPU because not all expressions can be replaced
  @Expression <AggregateExpression> stringindexeraggregator(org.apache.spark.ml.feature.StringIndexerAggregator@2f5fa317, Some(createexternalrow(RENT_NM#5829.toString, StructField(RENT_NM,StringType,true))), Some(interface org.apache.spark.sql.Row), Some(StructType(StructField(RENT_NM,StringType,true))), encodeusingserializer(input[0, java.lang.Object, true], true), decodeusingserializer(input[0, binary, true], Array[org.apache.spark.util.collection.OpenHashMap], true), encodeusingserializer(input[0, java.lang.Object, true], true), BinaryType, true, 0, 0) could run on GPU
    ! <Comple

In [15]:
from synapse.ml.automl import *
from pyspark.ml.regression import RandomForestRegressor
from synapse.ml.train import TrainRegressor

# 알고리즘 설정
rf = RandomForestRegressor()
lgbm = LightGBMRegressor()

smlmodels = [rf, lgbm]
mmlmodels = [TrainRegressor(model=model, labelCol="RENT_CNT") for model in smlmodels]

In [16]:
# 각 알고리즘의 하이퍼파라미터 설정
paramBuilder  = (
    HyperparamBuilder() 
    .addHyperparam(rf, rf.numTrees, DiscreteHyperParam([10, 20])) 
    .addHyperparam(rf, rf.maxDepth, DiscreteHyperParam([5, 10])) 
    .addHyperparam(lgbm, lgbm.learningRate, RangeHyperParam(0.1, 0.3)) 
    .addHyperparam(lgbm, lgbm.numLeaves, DiscreteHyperParam([31, 50])) 
)

searchSpace = paramBuilder.build()
randomSpace = RandomSpace(searchSpace)
print(searchSpace)
print(randomSpace)

dict_items([(Param(parent='RandomForestRegressor_86818ef43856', name='numTrees', doc='Number of trees to train (>= 1).'), (RandomForestRegressor_86818ef43856, <synapse.ml.automl.HyperparamBuilder.DiscreteHyperParam object at 0x7fa2b01fdf10>)), (Param(parent='RandomForestRegressor_86818ef43856', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'), (RandomForestRegressor_86818ef43856, <synapse.ml.automl.HyperparamBuilder.DiscreteHyperParam object at 0x7fa2b0d36780>)), (Param(parent='LightGBMRegressor_16812e0fb1d0', name='learningRate', doc='Learning rate or shrinkage rate'), (LightGBMRegressor_16812e0fb1d0, <synapse.ml.automl.HyperparamBuilder.RangeHyperParam object at 0x7fa28ccf3cb0>)), (Param(parent='LightGBMRegressor_16812e0fb1d0', name='numLeaves', doc='Number of leaves'), (LightGBMRegressor_16812e0fb1d0, <synapse.ml.automl.HyperparamBuilder.DiscreteHyperParam object at 0x7f

In [17]:
# AutoML 설정
automl = TuneHyperparameters(
    evaluationMetric="rmse",            ## 검증방식 설정
    models=mmlmodels,                   ## 모델 알고리즘 설정
    numFolds=2,                         ## 교차 검증 폴드 수
    numRuns=len(mmlmodels),             ## 랜덤 서치 횟수
    parallelism=1,                      ## 병렬로 실행할 모델의 수
    paramSpace=randomSpace.space(),     ## 하이퍼파라미터 공간을 정의하는 객체
    seed=42                             ## 난수 로직 설정
)

# 모델 학습
fittedModel = automl.fit(train)

24/01/03 10:52:08 WARN GpuOverrides: 
! <DeserializeToObjectExec> cannot run on GPU because not all expressions can be replaced; GPU does not currently support the operator class org.apache.spark.sql.execution.DeserializeToObjectExec
  ! <CreateExternalRow> createexternalrow(staticinvoke(class java.lang.Integer, ObjectType(class java.lang.Integer), valueOf, RENT_CNT#190, true, false, true), newInstance(class org.apache.spark.ml.linalg.VectorUDT).deserialize, StructField(RENT_CNT,IntegerType,true), StructField(features,org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7,true)) cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.objects.CreateExternalRow
    ! <StaticInvoke> staticinvoke(class java.lang.Integer, ObjectType(class java.lang.Integer), valueOf, RENT_CNT#190, true, false, true) cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.objects.StaticInvoke
  

[LightGBM] [Info] Saving data reference to binary buffer


24/01/03 10:54:44 WARN GpuOverrides: 
! <SerializeFromObjectExec> cannot run on GPU because not all expressions can be replaced; GPU does not currently support the operator class org.apache.spark.sql.execution.SerializeFromObjectExec
  @Expression <Alias> encodeusingserializer(input[0, java.lang.Object, true], true) AS value#9510 could run on GPU
    ! <EncodeUsingSerializer> encodeusingserializer(input[0, java.lang.Object, true], true) cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.objects.EncodeUsingSerializer
      ! <BoundReference> input[0, java.lang.Object, true] cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.BoundReference
  ! <MapPartitionsExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.MapPartitionsExec; not all expressions can be replaced
    !Expression <AttributeReferenc

In [18]:
# 최적화된 모델의 세부 정보
fittedModel.getBestModelInfo()

'bootstrap: true, cacheNodeIds: false, checkpointInterval: 10, featureSubsetStrategy: auto, featuresCol: TrainRegressor_6a97b1b2d846_features, impurity: variance, labelCol: RENT_CNT, leafCol: , maxBins: 32, maxDepth: 10, maxMemoryInMB: 256, minInfoGain: 0.0, minInstancesPerNode: 1, minWeightFractionPerNode: 0.0, numTrees: 20, predictionCol: prediction, seed: 3470451076654664240, subsamplingRate: 1.0'

In [19]:
# 최적화된 모델 보기
fittedModel.getBestModel()

TrainedRegressorModel_e24cebbbc598

In [20]:
# 최적화된 모델 저장
bestModel = fittedModel.getBestModel()

In [21]:
from synapse.ml.train import ComputeModelStatistics

prediction = bestModel.transform(test)

# ComputeModelStatistics를 사용하여 검증
metrics = ComputeModelStatistics().transform(prediction)
metrics.limit(10).toPandas()

24/01/03 11:01:57 WARN GpuOverrides: 
!Exec <ProjectExec> cannot run on GPU because unsupported data types in input: org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 [TrainRegressor_6a97b1b2d846_features#13347]; not all expressions can be replaced
  @Expression <Alias> UDF(TrainRegressor_6a97b1b2d846_features#13347) AS prediction#13377 could run on GPU
    !Expression <ScalaUDF> UDF(TrainRegressor_6a97b1b2d846_features#13347) cannot run on GPU because param expression AttributeReference TrainRegressor_6a97b1b2d846_features#13347 (org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 is not supported); neither UDF implemented by class org.apache.spark.ml.regression.RandomForestRegressionModel$$Lambda$5483/0x00007f138d424bd0 provides a GPU implementation, nor the conf `spark.rapids.sql.rowBasedUDF.enabled` is enabled
      !Expression <AttributeReference> TrainRegressor_6a97b1b2d846_features#13347 cannot run on GPU because expression AttributeReference TrainRegressor_6a97b1b2d846_features#13347 pro

,mean_squared_error,root_mean_squared_error,R^2,mean_absolute_error
0,136.120613,11.667074,0.923208,6.268358


In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

# 검증
evaluator_automl = RegressionEvaluator(labelCol="RENT_CNT", predictionCol="prediction", metricName="rmse")
# evaluator_automl = RegressionEvaluator(labelCol="RENT_CNT", predictionCol="prediction", metricName="r2")

rmse_automl = evaluator_automl.evaluate(prediction)
# r2_automl = evaluator.evaluate(prediction)

24/01/03 11:02:04 WARN GpuOverrides: 
! <DeserializeToObjectExec> cannot run on GPU because not all expressions can be replaced; GPU does not currently support the operator class org.apache.spark.sql.execution.DeserializeToObjectExec
  ! <CreateExternalRow> createexternalrow(staticinvoke(class java.lang.Double, ObjectType(class java.lang.Double), valueOf, prediction#14713, true, false, true), staticinvoke(class java.lang.Double, ObjectType(class java.lang.Double), valueOf, RENT_CNT#14714, true, false, true), staticinvoke(class java.lang.Double, ObjectType(class java.lang.Double), valueOf, 1.0#14715, true, false, true), StructField(prediction,DoubleType,false), StructField(RENT_CNT,DoubleType,true), StructField(1.0,DoubleType,false)) cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.catalyst.expressions.objects.CreateExternalRow
    ! <StaticInvoke> staticinvoke(class java.lang.Double, ObjectType(class java.lang.Double), valueOf, prediction

In [23]:
rmse_automl

11.667073894568436